In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time


# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [2]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [3]:
def welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array, v_E_array, y_el, y_in, \
                a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    # Variable indices:
    # In full:
    # y_el indices: (group, edge, "lane", time)
    # y_in indices: (group, edge, "lane", time)
    # Here:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")
    
    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]
    
    ## Compute lane flows:
    
    # Express lane (ex):
    x_ex = 0.0
    x_ex += sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1])
    x_ex += sum(y_in[(g, 0)] for g in range(num_in))
    
    # General purpose lane (gp):
    x_gp = 0.0 
    x_gp += sum(y_el[(g, 2)] for g in range(num_el))
    x_gp += sum(y_in[(g, 1)] for g in range(num_in))
    
    ell_ex = a_ex[4] * (x_ex ** 4) + a_ex[3] * (x_ex ** 3) + a_ex[2] * (x_ex ** 2) \
                + a_ex[1] * x_ex + a_ex[0]
    ell_gp = a_gp[4] * (x_gp ** 4) + a_gp[3] * (x_gp ** 3) + a_gp[2] * (x_gp ** 2) \
                + a_gp[1] * x_gp + a_gp[0]
        
    obj_E = tau * sum(y_el[(g, 1)] for g in range(num_el)) \
                + sum(y_el[(g, k)] * v_E_array[g] for g in range(num_el) for k in [0, 1]) * ell_ex \
                + sum(y_el[(g, 2)] * v_E_array[g] for g in range(num_el)) * ell_gp
    obj_I = tau * sum(y_in[(g, 0)] for g in range(num_in)) \
                + sum(y_in[(g, 0)] * v_I_array[g] for g in range(num_in)) * ell_ex \
                + sum(y_in[(g, 1)] * v_I_array[g] for g in range(num_in)) * ell_gp  
    obj_R = tau * sum(y_el[(g, 1)] for g in range(num_el)) \
                + tau * sum(y_in[(g, 0)] for g in range(num_in))
    
    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I
    
    print()
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print("welfare:", welfare)
    print()

    return welfare

    

In [4]:
def proj_tau_B_11(tau, B, tau_max = 1.0, B_max = 1.0):
    
    tau_feas = cp.Variable(1)
    B_feas = cp.Variable(1)
    
    func = (tau_feas - tau)**2 + (B_feas - B)**2

    objective = cp.Minimize(func)
    
    constraints = []
#     constraints += [0 <= tau_feas <= 1]
#     constraints += [0 <= B_feas <= 1]
    constraints += [tau_feas >= 0.0]
    constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    constraints += [B_feas <= B_max]
    constraints += [B_feas <= tau_feas]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas.value, 0.0), decimals=4)


# Chinmay's Algorithm:

In [7]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [8]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0
def solve_CBCP_direct(tau, B, v_I_array, v_E_array, \
                         a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
                         a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):
    
    # Variables:
    # y_el indices: (group, "lane")
    # y_in indices: (group, "lane")

    num_el = v_E_array.shape[0]
    num_in = v_I_array.shape[0]

    # Variables:
    y_el = {}
    for g in range(num_el):
        for k in [0, 1, 2]:
            y_el[(g, k)] = cp.Variable(1)
            
    y_in = {}
    for g in range(num_in):
        for k in [0, 1]:
            y_in[(g, k)] = cp.Variable(1)
            
    x_ex = cp.Variable(1)
    x_gp = cp.Variable(1)
    
    # Objective:
    func = 0.0
    
    func += 1/5 * a_ex[4] * cp.power(x_ex, 5)
    func += 1/4 * a_ex[3] * cp.power(x_ex, 4)
    func += 1/3 * a_ex[2] * cp.power(x_ex, 3)
    func += 1/2 * a_ex[1] * cp.power(x_ex, 2)
    func += a_ex[0] * x_ex
    func += 1/5 * a_gp[4] * cp.power(x_gp, 5)
    func += 1/4 * a_gp[3] * cp.power(x_gp, 4)
    func += 1/3 * a_gp[2] * cp.power(x_gp, 3)
    func += 1/2 * a_gp[1] * cp.power(x_gp, 2)
    func += a_gp[0] * x_gp
    
    func += sum(y_el[(g, 1)] / v_E_array[g] for g in range(num_el)) * tau
    func += sum(y_in[(g, 0)] / v_I_array[g] for g in range(num_in)) * tau

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y_el[(g, k)] >= 0.0 for g in range(num_el) for k in [0, 1, 2]]
    constraints += [y_in[(g, k)] >= 0.0 for g in range(num_in) for k in [0, 1]]
    
    constraints += [sum(y_el[(g, k)] for k in [0, 1, 2] ) == 1.0 for g in range(num_el)]
    constraints += [sum(y_in[(g, k)] for k in [0, 1] ) == 1.0 for g in range(num_in)]
    
    constraints += [x_ex == sum(y_el[(g, k)] for g in range(num_el) for k in [0, 1]) \
                               + sum(y_in[(g, 0)] for g in range(num_in))]
    constraints += [x_gp == sum(y_el[(g, 2)] for g in range(num_el)) \
                               + sum(y_in[(g, 1)] for g in range(num_in))]
    
    constraints += [y_el[(g, 0)] * tau <= B for g in range(num_el)]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
    result = prob.solve()
    
    # Extract Values:
    y_el_values = {}
    for g in range(num_el):
        for k in [0, 1, 2]:
            y_el_values[(g, k)] = y_el[(g, k)].value[0]
            
    y_in_values = {}
    for g in range(num_in):
        for k in [0, 1]:
            y_in_values[(g, k)] = y_in[(g, k)].value[0]

    return y_el_values, y_in_values


In [11]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0, 1.2, 1.4])
v_E_array = np.array([0.6, 0.8, 0.9])
flow_per_group = 1.0
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
# y_init = np.array([0.0, 0.05, 0.95, 0.95, 0.05])

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 2.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# tau_max, B_max = 1.0, 1.0
tau_max = flow_per_group * (v_I_array.shape[0] + v_E_array.shape[0])
B_max = tau_max

d = 2
num_iters = 1000
tau = np.zeros(num_iters)
tau_perturbed = np.zeros(num_iters)
B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 3.0
delta_bar = 3.0

welfare_list = []

# tau[0] = tau_max * 0.8
# B[0] = B_max * 0.2

tau[0] = tau_max * 0.7
B[0] = B_max * 0.3

# tau[0] = tau_max * 0.6
# B[0] = B_max * 0.4


for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(2)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
    print("w_i:", w_i)
    tau_perturbed[i] = tau[i] + delta[i] * w_i[0]
    B_perturbed[i] = B[i] + delta[i] * w_i[1]
    
#     if tau_perturbed[i] < B_perturbed[i] or tau_perturbed[i] < 0 or B_perturbed[i] < 0:
    tau_perturbed[i], B_perturbed[i] = proj_tau_B_11(tau_perturbed[i], B_perturbed[i], \
                                                    tau_max = tau_max, B_max = B_max)
    
    print("tau[i]:", tau[i])
    print("B[i]:", B[i])
    print("tau_perturbed[i]:", tau_perturbed[i])
    print("B_perturbed[i]:", B_perturbed[i])


    y_el_values, y_in_values = solve_CBCP_direct(tau = tau[i], B = B[i], v_I_array = v_I_array, \
                                                 v_E_array = v_E_array, a_ex = a, a_gp = a)

    print()
    print("y_el_values:", y_el_values)
    print("y_in_values:", y_in_values)
    print()

#     print()
#     print("New solve_CBCP_iter_11 call to solve_CBCP (perturbed):")
#     print()

    y_el_perturbed_values, y_in_perturbed_values = solve_CBCP_direct(tau = tau_perturbed[i], B = B_perturbed[i], \
                                                                     v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
    
#     print("y_el:", y_el)
#     print("y_in:", y_in)
#     print("y_el_perturbed:", y_el_perturbed)
#     print("y_in_perturbed:", y_in_perturbed)
    
    welfare = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau[i], v_I_array = v_I_array, v_E_array = v_E_array, \
                          y_el = y_el_values, y_in = y_in_values, a_ex = a, a_gp = a)
    welfare_perturbed = welfare_obj(lambda_E, lambda_R, lambda_I, tau = tau_perturbed[i], v_I_array = v_I_array, v_E_array = v_E_array, \
                                    y_el = y_el_perturbed_values, y_in = y_in_perturbed_values, a_ex = a, a_gp = a)
    
    welfare_list.append(welfare)
    
        ## TODO: Edit below to use the fact that creatively, CVXPY can handle multiple-dimensional n ,\
    ## (e.g., 3d or 4d) arrays:\n,
    
#     def welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I, v_E, y_el, y_in, \
#                 a_ex = np.array([0.0, 0.0, 0.0, 0.0, 1.0]), \
#                 a_gp = np.array([0.0, 0.0, 0.0, 0.0, 1.0]))
    
    print("welfare:", welfare)
    print("welfare_perturbed:", welfare_perturbed)
#     print("tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare):\n", tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare))
    
    tau[i+1] = tau[i] - eta[i] * (d/delta[i]) * w_i[0] * (welfare_perturbed - welfare)
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[1] * (welfare_perturbed - welfare)
    tau[i+1], B[i+1] = proj_tau_B_11(tau[i+1], B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = tau[i-diffs_num_cols : i-1] - tau[i-diffs_num_cols+1 : i]
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[0:10]:", tau[0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:", tau_diffs)
        print("B_diffs:", B_diffs)
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)




Iter: 0
w_i: [ 0.15681114 -0.98762861]
tau[i]: 4.199999999999999
B[i]: 1.7999999999999998
tau_perturbed[i]: 4.5326
B_perturbed[i]: 0.0

y_el_values: {(0, 0): 0.42857141782322644, (0, 1): 1.4966304195951352e-08, (0, 2): 0.5714285672104693, (1, 0): 0.4285714208275619, (1, 1): 2.5652277191312578e-08, (1, 2): 0.5714285535201609, (2, 0): 0.42857142307206897, (2, 1): 4.049808211004091e-08, (2, 2): 0.5714285364298489}
y_in_values: {(0, 0): 5.932492508353704e-08, (0, 1): 0.9999999406750749, (1, 0): 0.6980772671871015, (1, 1): 0.3019227328128985, (2, 0): 0.9999998033974922, (2, 1): 1.966025078186952e-07}


obj_E: 186.90781765817633
obj_R: 7.131924286309964
obj_I: 297.2505717301261
welfare: 669.6398221892168


obj_E: 191.57046191445886
obj_R: 13.502632674892645
obj_I: 297.1198420394499
welfare: 677.5602393333891

welfare: 669.6398221892168
welfare_perturbed: 677.5602393333891

Iter: 1
w_i: [-0.13868154  0.99033703]
tau[i]: 1.0
B[i]: 1.0
tau_perturbed[i]: 1.7596
B_perturbed[i]: 1.7596

y_el_valu

tau_diffs: [-0.0062  0.      0.0062 -0.    ]
B_diffs: [-0.0062  0.      0.0062  0.    ]

Iter: 10
w_i: [-0.37123409  0.92853931]
tau[i]: 0.0
B[i]: 0.0
tau_perturbed[i]: 0.3246
B_perturbed[i]: 0.3246

y_el_values: {(0, 0): -0.0, (0, 1): 0.5000000001051168, (0, 2): 0.49999999989488314, (1, 0): -0.0, (1, 1): 0.5000000001051162, (1, 2): 0.49999999989488375, (2, 0): -0.0, (2, 1): 0.5000000001051166, (2, 2): 0.49999999989488336}
y_in_values: {(0, 0): 0.500000000105117, (0, 1): 0.49999999989488303, (1, 0): 0.5000000001051168, (1, 1): 0.49999999989488325, (2, 0): 0.5000000001051168, (2, 1): 0.49999999989488325}


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.26557295607256
obj_R: 2.4310088020727818e-06
obj_I: 291.653900990985
welfare: 664.1850464169283

welfare: 664.2000000000003
welfare_perturbed: 664.1850464169283
tau_diffs: [ 0.      0.0062 -0.      0.    ]
B_diffs: [0.     0.0062 0.     0.    ]

Iter: 11
w_i: [ 0.60171275 -0.79871257]

tau_diffs: [-0.0526  0.4433 -0.0079 -0.006 ]
B_diffs: [ 0.045   0.3359 -0.0079 -0.006 ]

Iter: 20
w_i: [0.36868572 0.92955411]
tau[i]: -0.0
B[i]: -0.0
tau_perturbed[i]: 0.6432
B_perturbed[i]: 0.6432

y_el_values: {(0, 0): -0.0, (0, 1): 0.5000000001051168, (0, 2): 0.49999999989488314, (1, 0): -0.0, (1, 1): 0.5000000001051162, (1, 2): 0.49999999989488375, (2, 0): -0.0, (2, 1): 0.5000000001051166, (2, 2): 0.49999999989488336}
y_in_values: {(0, 0): 0.500000000105117, (0, 1): 0.49999999989488303, (1, 0): 0.5000000001051168, (1, 1): 0.49999999989488325, (2, 0): 0.5000000001051168, (2, 1): 0.49999999989488325}


obj_E: 186.30000000000007
obj_R: -0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.27191545575118
obj_R: 2.038397695440021e-06
obj_I: 291.64396920945137
welfare: 664.1877997132742

welfare: 664.2000000000003
welfare_perturbed: 664.1877997132742
tau_diffs: [ 0.4433 -0.0079 -0.006  -0.001 ]
B_diffs: [ 0.3359 -0.0079 -0.006  -0.0004]

Iter: 21
w_i: [-0.27660901  0.960


obj_E: 186.30000000000007
obj_R: -0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.27385118390634
obj_R: 1.865579031800457e-06
obj_I: 291.6409378650077
welfare: 664.1886398597046

welfare: 664.2000000000003
welfare_perturbed: 664.1886398597046
tau_diffs: [-0.      0.     -0.0068  0.    ]
B_diffs: [ 0.      0.     -0.0068  0.    ]

Iter: 30
w_i: [0.41485195 0.90988893]
tau[i]: 0.0015
B[i]: 0.0015
tau_perturbed[i]: 0.597
B_perturbed[i]: 0.597

y_el_values: {(0, 0): 0.999553901077918, (0, 1): 5.64605423432063e-05, (0, 2): 0.00038963837973882143, (1, 0): 0.9995230045889577, (1, 1): 5.395471110535325e-05, (1, 2): 0.0004230406999369381, (2, 0): 0.9995092612644606, (2, 1): 4.588226616005761e-05, (2, 2): 0.0004448564693793401}
y_in_values: {(0, 0): 7.149709877452004e-05, (0, 1): 0.9999285029012255, (1, 0): 0.00017675756367874218, (1, 1): 0.9998232424363213, (2, 0): 0.0010254394942256218, (2, 1): 0.9989745605057744}


obj_E: 186.30401067043024
obj_R: 2.1449875144312516e-06



y_el_values: {(0, 0): -0.0, (0, 1): 0.5000000001051168, (0, 2): 0.49999999989488314, (1, 0): -0.0, (1, 1): 0.5000000001051162, (1, 2): 0.49999999989488375, (2, 0): -0.0, (2, 1): 0.5000000001051166, (2, 2): 0.49999999989488336}
y_in_values: {(0, 0): 0.500000000105117, (0, 1): 0.49999999989488303, (1, 0): 0.5000000001051168, (1, 1): 0.49999999989488325, (2, 0): 0.5000000001051168, (2, 1): 0.49999999989488325}


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.27212666507882
obj_R: 1.7636857315874692e-06
obj_I: 291.64363799097777
welfare: 664.1878909683983

welfare: 664.2000000000003
welfare_perturbed: 664.1878909683983
tau_diffs: [-0.145  -0.0014  0.1464  0.    ]
B_diffs: [-0.145  -0.0014  0.1464  0.    ]

Iter: 40
w_i: [ 0.77744512 -0.62895078]
tau[i]: 0.0034
B[i]: 0.0034
tau_perturbed[i]: 0.6551
B_perturbed[i]: -0.0

y_el_values: {(0, 0): 0.9999197759054819, (0, 1): 1.664882395280287e-05, (0, 2): 6.35752705653303e-05, (1, 0): 0.9999


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.89547161959064
obj_R: 1.5410690603821398
obj_I: 292.2124958847208
welfare: 665.6952253118257

welfare: 664.2000000000003
welfare_perturbed: 665.6952253118257
tau_diffs: [-0.0963 -0.1467  0.2478 -0.    ]
B_diffs: [-0.0963 -0.1467  0.2478 -0.    ]

Iter: 49
w_i: [ 0.22357894 -0.97468583]
tau[i]: 0.0505
B[i]: 0.0505
tau_perturbed[i]: 0.2289
B_perturbed[i]: 0.0

y_el_values: {(0, 0): 0.9999458701228361, (0, 1): 1.9930596406592116e-06, (0, 2): 5.213681752325172e-05, (1, 0): 0.9999574702963302, (1, 1): 3.161283739379586e-06, (1, 2): 3.936841993042288e-05, (2, 0): 0.9999671237451537, (2, 1): 3.757593799497831e-06, (2, 2): 2.9118661046772383e-05}
y_in_values: {(0, 0): 6.635753284078838e-06, (0, 1): 0.9999933642467159, (1, 0): 9.712628587399408e-06, (1, 1): 0.9999902873714126, (2, 0): 1.1989645826604445e-05, (2, 1): 0.9999880103541734}


obj_E: 186.2770794712729
obj_R: 1.8811232263197757e-06
o


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003

welfare: 664.2000000000003
welfare_perturbed: 664.2000000000003
tau_diffs: [ 0.2388 -0.0088 -0.0041  0.0129]
B_diffs: [ 0.2013 -0.0088 -0.0041  0.0129]

Iter: 59
w_i: [-0.25378508  0.96726063]
tau[i]: 0.0
B[i]: 0.0
tau_perturbed[i]: 0.2719
B_perturbed[i]: 0.2719

y_el_values: {(0, 0): -0.0, (0, 1): 0.5000000001051168, (0, 2): 0.49999999989488314, (1, 0): -0.0, (1, 1): 0.5000000001051162, (1, 2): 0.49999999989488375, (2, 0): -0.0, (2, 1): 0.5000000001051166, (2, 2): 0.49999999989488336}
y_in_values: {(0, 0): 0.500000000105117, (0, 1): 0.49999999989488303, (1, 0): 0.5000000001051168, (1, 1): 0.49999999989488325, (2, 0): 0.5000000001051168, (2, 1): 0.49999999989488325}


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.27272948184773
obj_R: 1.804519303711348


y_el_values: {(0, 0): 0.9453370099883991, (0, 1): 4.892618854435149e-05, (0, 2): 0.05461406382305656, (1, 0): 0.9453082555182698, (1, 1): 0.00013495981455759892, (1, 2): 0.0545567846671726, (2, 0): 0.9452872245196638, (2, 1): 0.0002200560054122988, (2, 2): 0.05449271947492385}
y_in_values: {(0, 0): 0.00015246094796683707, (0, 1): 0.9998475390520332, (1, 0): 0.0002078132808551736, (1, 1): 0.9997921867191448, (2, 0): 0.16329544366816995, (2, 1): 0.83670455633183}


obj_E: 186.29826499967143
obj_R: 0.0009023281294802841
obj_I: 291.60356390294663
welfare: 664.1999134366636


obj_E: 186.30000000000007
obj_R: -0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003

welfare: 664.1999134366636
welfare_perturbed: 664.2000000000003
tau_diffs: [ 0.005   0.     -0.0019 -0.0036]
B_diffs: [ 0.005   0.     -0.0019 -0.0036]

Iter: 69
w_i: [-0.12414869 -0.99226363]
tau[i]: 0.0055
B[i]: 0.0052
tau_perturbed[i]: -0.0
B_perturbed[i]: 0.0

y_el_values: {(0, 0): 0.9453370099883991, (0, 1): 4.8926188544351

tau[i]: 0.0042
B[i]: 0.0042
tau_perturbed[i]: -0.0
B_perturbed[i]: -0.0

y_el_values: {(0, 0): 0.9999463351245114, (0, 1): 7.221335383444938e-06, (0, 2): 4.644354010518992e-05, (1, 0): 0.9999561773128104, (1, 1): 1.3302966457598672e-05, (1, 2): 3.0519720732011016e-05, (2, 0): 0.9999466082638779, (2, 1): 1.759119387510567e-05, (2, 2): 3.580054224696027e-05}
y_in_values: {(0, 0): 3.517561811361425e-05, (0, 1): 0.9999648243818864, (1, 0): 4.898884822734928e-05, (1, 1): 0.9999510111517727, (2, 0): 3.2043943157256116e-05, (2, 1): 0.9999679560568427}


obj_E: 186.30085573891924
obj_R: 6.481604019003495e-07
obj_I: 291.5986613306978
welfare: 664.2003726789042


obj_E: 186.30000000000007
obj_R: -0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003

welfare: 664.2003726789042
welfare_perturbed: 664.2000000000003
tau_diffs: [-4.9e-03 -4.8e-03 -1.0e-04  9.8e-03]
B_diffs: [-4.9e-03 -4.8e-03 -1.0e-04  9.8e-03]

Iter: 79
w_i: [0.6745555  0.73822414]
tau[i]: 0.0041
B[i]: 0.0041
tau_perturbed[i]: 0.


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 186.30000000000007
obj_R: -0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003

welfare: 664.2000000000003
welfare_perturbed: 664.2000000000003
tau_diffs: [ 0.0138 -0.0052  0.0002  0.0001]
B_diffs: [ 0.0138 -0.0052  0.0002  0.0001]

Iter: 88
w_i: [ 0.97937938 -0.20202976]
tau[i]: 0.0
B[i]: 0.0
tau_perturbed[i]: 0.6764
B_perturbed[i]: -0.0

y_el_values: {(0, 0): -0.0, (0, 1): 0.5000000001051168, (0, 2): 0.49999999989488314, (1, 0): -0.0, (1, 1): 0.5000000001051162, (1, 2): 0.49999999989488375, (2, 0): -0.0, (2, 1): 0.5000000001051166, (2, 2): 0.49999999989488336}
y_in_values: {(0, 0): 0.500000000105117, (0, 1): 0.49999999989488303, (1, 0): 0.5000000001051168, (1, 1): 0.49999999989488325, (2, 0): 0.5000000001051168, (2, 1): 0.49999999989488325}


obj_E: 186.30000000000007
obj_R: 0.0
obj_I: 291.6000000000001
welfare: 664.2000000000003


obj_E: 187.08313444833922
obj_R: 2.0270708585413932

In [12]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

first(welfare_list): 669.6398221892168
min(welfare_list): 663.8586503665125
max(welfare_list): 669.6398221892168
argmin_tau: 1.0
argmin_B: 0.9969


## Grid Search:

In [ ]:
# tau = 0.5
# B = 0.5
# v_I = 1.0
# v_E = 0.6
# a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

# y_el_point_5, y_in_point_5 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]),\
#                                                a_ex = a, a_gp = a)

# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                   y_el = y_el_point_5, y_in = y_in_point_5, \
#                                                   a_ex = a, a_gp = a)


In [ ]:
time_1 = time.time()

tau = 0.4
B = 0.3
v_I_array = np.array([1.0])
v_E_array = np.array([0.6])
a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])
num_iters_max = 5000
error_bound = 1E-3
diffs_num_cols = 5
y_init = np.array([0.0, 0.0, 1.0, 1.0, 0.0])
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 0.2, 1.0
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0


grid_size = 0.02
tau_arr = (np.arange(int(1/grid_size)) + 1) * grid_size
B_arr = (np.arange(int(1/grid_size)) + 1) * grid_size

welfare_obj_arr = np.ones((tau_arr.shape[0], B_arr.shape[0])) * 100
y_el_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 3))
y_in_arr = np.zeros((tau_arr.shape[0], B_arr.shape[0], 2))
welfare_obj_list = []

for tau_index, tau in enumerate(tau_arr):
    for B_index, B in enumerate(B_arr):
        if B < tau:
            print("tau:", tau)
            print("B:", B)
            y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
                = solve_CBCP_direct(tau = tau, B = B,\
                                   v_I_array = v_I_array, v_E_array = v_E_array, a_ex = a, a_gp = a)
            
#             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
            welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = v_I_array, v_E_array = v_E_array, \
                                                              y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
                                                              a_ex = a, a_gp = a)
        
            welfare_obj_list.append(welfare_obj_arr[tau_index][B_index])
            
            print()

time_2 = time.time()

print("Time:", time_2 - time_1)



In [ ]:
# tau = 0.5
# B = 0.4
# v_I = 1.0
# v_E = 0.6

# y_el_arr[tau_index, B_index, :], y_in_arr[tau_index, B_index, :] \
#                 = solve_CBCP_direct(tau = tau, B = B, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), a_ex = a, a_gp = a)
            
# #             def solve_CBCP_direct(tau, B, v_I, v_E, a = np.array([0.5, 1.0])):
            
# welfare_obj_arr[tau_index][B_index] = welfare_obj(lambda_E, lambda_R, lambda_I, tau, v_I_array = np.array([v_I]), v_E_array = np.array([v_E]), \
#                                                               y_el = y_el_arr[tau_index, B_index, :], y_in = y_in_arr[tau_index, B_index, :], \
#                                                               a_ex = a, a_gp = a)

In [ ]:
# welfare_obj_arr

argmin_indices_wrapped = np.where(welfare_obj_arr == min(welfare_obj_list))
argmin_indices = [argmin_indices_wrapped[0][0], argmin_indices_wrapped[1][0]]
# argmin_indices
argmin_tau = tau_arr[argmin_indices[0]]
argmin_B = B_arr[argmin_indices[1]]

print("argmin_tau:\n", np.round(argmin_tau, 2))
print("\nargmin_B:\n", np.round(argmin_B, 2))

welfare_obj_arr[argmin_indices[0], argmin_indices[1]]


In [ ]:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# plt.imshow(welfare_obj_arr.T, vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 
plt.imshow(welfare_obj_arr.T, extent=[np.min(tau_arr), np.max(tau_arr), np.min(B_arr), np.max(B_arr)], \
           vmin = min(welfare_obj_list), vmax = max(welfare_obj_list), origin='lower') 

plt.colorbar() 
plt.xlabel("Toll")
plt.ylabel("Budget")
# plt.xticks(x_positions, x_labels)

## Test:

In [ ]:
# Test:

grad = np.array([3.11430535, 1.501, 1.501, 2.46858321, 1.501])

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_el_var = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_in_var = cp.Variable(2)

objective = cp.Minimize(grad[0:3] @ y_el_var + grad[3:] @ y_in_var)

constraints = []
constraints += [y_el_var >= 0, y_in_var >= 0]
constraints += [cp.sum(y_el_var) == 1.0, cp.sum(y_in_var) == 1.0]
constraints += [y_el_var[1] * tau <= B]

prob = cp.Problem(objective, constraints)
result = prob.solve()

print("grad:", grad)
print("y_el_var.value:", y_el_var.value)
# print("y_el_var_current:", y_el_var_current)
print("y_in_var.value:", y_in_var.value)
# print("y_in_var_current:", y_in_var_current)
print()

# y_el_var_current = y_el_var_current + 2/(k+2) * (y_el_var.value - y_el_var_current)
# y_in_var_current = y_in_var_current + 2/(k+2) * (y_in_var.value - y_in_var_current)

# y_iters[0:3, k] = y_el_var_current
# y_iters[3:, k] = y_in_var_current

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()

# Solver=SCS,verbose=False

In [ ]:
# y_el_var
# y_in_var
# np.hstack((y_el_var, y_in_var))

# Scratch Work:

In [3]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


x.value: [-0.  1.]
y.value: [-0.  1.]



# Scratch Work:

In [ ]:
# # Test:

# Variables:

# y_el: \hat y_1 E, \tilde y_1 E, y_2 E
y_elig = cp.Variable(3)
# y_in: y_1 I, y_2 I
y_inel = cp.Variable(2)


# Objective:

a = np.array([0.0, 0.0, 0.0, 0.0, 1.0])

tau = 0.5
B = 0.4

func = 0.0
func += 1/5 * a[4] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 5)
func += 1/4 * a[3] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 4)
func += 1/3 * a[2] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 3)
func += 1/2 * a[1] * cp.power(y_elig[0] + y_elig[1] + y_inel[0], 2)
func += a[0] * (y_elig[0] + y_elig[1] + y_inel[0])
func += y_inel[0] * tau / v_I + y_elig[0] * tau / v_E
func += 1/5 * a[4] * cp.power(y_elig[2] + y_inel[1], 5)
func += 1/4 * a[3] * cp.power(y_elig[2] + y_inel[1], 4)
func += 1/3 * a[2] * cp.power(y_elig[2] + y_inel[1], 3)
func += 1/2 * a[1] * cp.power(y_elig[2] + y_inel[1], 2)
func += a[0] * (y_elig[2] + y_inel[1])

objective = cp.Minimize(func)


# Constraints:

constraints = []
constraints += [y_elig >= 0, y_inel >= 0]
constraints += [cp.sum(y_elig) == 1, cp.sum(y_inel) == 1]
constraints += [y_elig[1] * tau <= B]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("y_elig.value:", np.round(y_elig.value, 4) )
print("y_inel.value:", np.round(y_inel.value, 4) )
print()



# power(x, p)



In [ ]:
x1 = 1
x2 = 2
print("x1:", x1, ", x2:", x2)

## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:

func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)
